# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()
#city_data_df.count()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,carnarvon,-28.2695,98.9508,18.29,64,80,2.98,1691255258
1,1,akureyri,75.7387,-17.7677,0.73,83,100,3.95,1691255258
2,2,kulia village,-6.0489,177.7185,28.55,76,47,5.65,1691255258
3,3,badger,85.9124,-143.0261,-0.96,79,38,2.92,1691255259
4,4,stanley,-74.0755,-53.5453,-33.02,100,100,2.24,1691255259


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
myidealtemp_df = city_data_df[city_data_df["Max Temp"] >= 24]

myidealtemp_df.head()
#myidealtemp_df.count()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
2,2,kulia village,-6.0489,177.7185,28.55,76,47,5.65,1691255258
10,10,gurupi,-11.5350,-50.0480,38.46,19,1,1.85,1691255260
13,13,tyre,33.6403,34.0250,28.40,70,0,6.18,1691255260
14,14,rongelap,23.3427,166.1161,26.85,78,69,5.80,1691255260
22,22,hawaiian paradise park,12.0394,-151.9149,26.36,75,70,8.09,1691255261


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
myidealtemp_df = city_data_df[city_data_df["Max Temp"] >= 24]


# Drop any rows with null values
myidealtemp_clean_df = myidealtemp_df.dropna(axis=1)

# Display sample data
myidealtemp_clean_df.head()
#myidealtemp_clean_df.count()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
2,2,kulia village,-6.0489,177.7185,28.55,76,47,5.65,1691255258
10,10,gurupi,-11.5350,-50.0480,38.46,19,1,1.85,1691255260
13,13,tyre,33.6403,34.0250,28.40,70,0,6.18,1691255260
14,14,rongelap,23.3427,166.1161,26.85,78,69,5.80,1691255260
22,22,hawaiian paradise park,12.0394,-151.9149,26.36,75,70,8.09,1691255261


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = myidealtemp_clean_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
2,2,kulia village,-6.0489,177.7185,28.55,76,47,5.65,1691255258,
10,10,gurupi,-11.5350,-50.0480,38.46,19,1,1.85,1691255260,
13,13,tyre,33.6403,34.0250,28.40,70,0,6.18,1691255260,
14,14,rongelap,23.3427,166.1161,26.85,78,69,5.80,1691255260,
22,22,hawaiian paradise park,12.0394,-151.9149,26.36,75,70,8.09,1691255261,


In [29]:
# latitude = hotel_df["Lat"]
# print(latitude)
# longitude = hotel_df["Lng"]
# print(longitude)

# params["filter"] = f"circle:{longitude},{latitude},{radius}"
# params["bias"] = f"proximity:{longitude},{latitude}"

# params

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
##Zoom = API and Geospatial Data Visual... @ 1:13:23 the Nearest Restaurants in Madrid activity

# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "format": "json",
    "text": hotel_df['City']
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df["Lat"]
    longitude = hotel_df["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    ## Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    ## Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    ## Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

## Display sample data
hotel_df

Starting hotel search
kulia village - nearest hotel: No hotel found
gurupi - nearest hotel: No hotel found
tyre - nearest hotel: No hotel found
rongelap - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
saksaul'skiy - nearest hotel: No hotel found
mocorito - nearest hotel: No hotel found
toga village - nearest hotel: No hotel found
priozersk - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found
roxas - nearest hotel: No hotel found
jiazi - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
zouerate - nearest hotel: No hotel found
duba - nearest hotel: No hotel found
turpan - nearest hotel: No hotel found
champerico - nearest hotel: No hotel found
port hedland - nearest hotel: No hotel found
touros - nearest hotel: No hotel found
taiohae - nearest hotel: No hotel found
al-hamdaniya - nearest hotel: No hotel found
hithadhoo - nearest hotel: No hotel found
holualoa - nearest hotel: No hotel found
cockburn t

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.5,
    color = "Hotel Name"
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Max Temp)